# Capstone Project - The Battle of Neighborhoods

# ------------------------------------------------------

## Introduction

##### In this project, we will try to cluster the cities of my own country Morocco based on their characteristics. In order to know the similar cities and to recommend new cities from people's preferences

## Data

##### The data that we'll be using is constructed of the cities names and the characteristics of each city based on the Foursquare location data

#### Import Libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

from bs4 import BeautifulSoup

print('Libraries imported.')

Libraries imported.


#### Get the liste of Moroccan cities

In [2]:
URL_Moroccan_Cities = 'https://en.wikipedia.org/wiki/List_of_cities_in_Morocco'
# Load article, turn into soup and get the <table>s
url = requests.get(URL_Moroccan_Cities).text
soup = BeautifulSoup(url,'lxml')
My_table = soup.find('table',{'class':'wikitable sortable'})
#My_table

for table in My_table:
    ths = My_table.find_all('th')
    headers = [th.text.strip() for th in ths]

#Correct headers
headers = ['Rank', 'City', 'Population', 'Region']    

i=1;
for tr in My_table.find_all('tr'):
    tds = tr.find_all('td')
    row = [td.text.strip() for td in tds]
    if i:
        col = row
        i=0
    else :
        col.append(row)

df = pd.DataFrame(col)
df.columns = headers

#Remove [a], [b].... from cities
df=df.replace('\[.\]','',regex=True)

#Drop renk column
df.drop(['Rank'], axis = 1, inplace = True)

df.head()
df.loc[df.index.max() + 1] = ['Laayoune', 217732, 'Sahara']
df.loc[df.index.max() + 1] = ['Dakhla', 106277, 'Sahara']
df.head()

,City,Population,Region
0,Casablanca,"3,359,818",Casablanca-Settat
1,Fez,"1,112,072",Fès-Meknès
2,Tangier,"947,952",Tanger-Tetouan-Al Hoceima
3,Marrakesh,"928,850",Marrakesh-Safi
4,Salé,"890,403",Rabat-Salé-Kénitra


#### Get latitude and longitude from address

In [3]:
import sys
import time

address = df['City']
geolocator = Nominatim(user_agent="foursquare_agent")
df['Latitude'] = ""
df['Longitude'] = ""

location = None
for city in address :
    while location is None:
        location = geolocator.geocode('{}, Morocco'.format(city))
    latitude = location.latitude
    longitude = location.longitude
    df['Latitude'].loc[df.index[df['City'] == city]] = latitude
    df['Longitude'].loc[df.index[df['City'] == city]] = longitude
    location = None
    print(city, ' : OK !')

print('Done !')
df.head()


Casablanca  : OK !
Fez  : OK !
Tangier  : OK !
Marrakesh  : OK !
Salé  : OK !
Meknes  : OK !
Rabat  : OK !
Oujda  : OK !
Kenitra  : OK !
Agadir  : OK !
Tetouan  : OK !
Temara  : OK !
Safi  : OK !
Mohammedia  : OK !
Khouribga  : OK !
El Jadida  : OK !
Beni Mellal  : OK !
Aït Melloul  : OK !
Nador  : OK !
Dar Bouazza  : OK !
Taza  : OK !
Settat  : OK !
Berrechid  : OK !
Khemisset  : OK !
Inezgane  : OK !
Ksar El Kebir  : OK !
Larache  : OK !
Guelmim  : OK !
Khenifra  : OK !
Berkane  : OK !
Taourirt  : OK !
Bouskoura  : OK !
Fquih Ben Salah  : OK !
Dcheira El Jihadia  : OK !
Oued Zem  : OK !
El Kelaa Des Sraghna  : OK !
Sidi Slimane  : OK !
Errachidia  : OK !
Guercif  : OK !
Oulad Teima  : OK !
Ben Guerir  : OK !
Tifelt  : OK !
Lqliaa  : OK !
Taroudant  : OK !
Sefrou  : OK !
Essaouira  : OK !
Fnideq  : OK !
Sidi Kacem  : OK !
Tiznit  : OK !
Tan-Tan  : OK !
Ouarzazate  : OK !
Souk El Arbaa  : OK !
Youssoufia  : OK !
Lahraouyine  : OK !
Martil  : OK !
Ain Harrouda  : OK !
Suq as-Sabt Awlad 

,City,Population,Region,Latitude,Longitude
0,Casablanca,"3,359,818",Casablanca-Settat,33.5951,-7.61878
1,Fez,"1,112,072",Fès-Meknès,34.0347,-5.01619
2,Tangier,"947,952",Tanger-Tetouan-Al Hoceima,35.7771,-5.80379
3,Marrakesh,"928,850",Marrakesh-Safi,31.6258,-7.98916
4,Salé,"890,403",Rabat-Salé-Kénitra,34.0157,-6.7568


#### Visualize the map

In [4]:
location = geolocator.geocode('Morocco')
latitude = location.latitude
longitude = location.longitude
# create map of New York using latitude and longitude values
map_Morocco = folium.Map(location=[latitude, longitude], zoom_start=5)

# add markers to map
for lat, lng, city in zip(df['Latitude'], df['Longitude'], df['City']):
    label = '{} : {}, {} '.format(city, lat, lng)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='Green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Morocco)  
    
map_Morocco

#### Define Foursquare Credentials and Version

In [5]:
CLIENT_ID = 'JDL4X53XBSDRDHG5H4CYLYEFJQYS5AYSFA5LTJA4VP3PXV3T' # your Foursquare ID
CLIENT_SECRET = 'AQA54H5YE3CE02SONVFCCBNV4K5CFCHGO3UT1ZVPQ3WVWOLG' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
radius = 1000
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: JDL4X53XBSDRDHG5H4CYLYEFJQYS5AYSFA5LTJA4VP3PXV3T
CLIENT_SECRET:AQA54H5YE3CE02SONVFCCBNV4K5CFCHGO3UT1ZVPQ3WVWOLG


In [6]:
df_venues = pd.DataFrame(columns=['City', 'Population','name', 'categories', 'lat', 'lng'])

for lat, lng, city, population in zip(df['Latitude'], df['Longitude'], df['City'], df['Population']):
    # Create the explore url
    latitude = lat
    longitude = lng
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION)


    # Send the GET Request and examine the results
    results = requests.get(url).json()


    # Get relevant part of JSON and transform it into a pandas dataframe

    # function that extracts the category of the venue
    def get_category_type(row):
        try:
            categories_list = row['categories']
        except:
            categories_list = row['venue.categories']

        if len(categories_list) == 0:
            return None
        else:
            return categories_list[0]['name']

    venues = results['response']['groups'][0]['items']

    nearby_venues = json_normalize(venues) # flatten JSON

    # filter columns
    filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
    nearby_venues =nearby_venues.loc[:, filtered_columns]

    # filter the category for each row
    nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

    # clean columns
    nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
    
    nearby_venues['City'] = ""
    nearby_venues['Population'] = ""
    nearby_venues = nearby_venues[['City', 'Population','name', 'categories', 'lat', 'lng']]
    for i in range(0,nearby_venues.index.max()+1):
        nearby_venues['City'].iloc[i] = city
        nearby_venues['Population'].iloc[i] = population
    
    df_venues = df_venues.append(nearby_venues, ignore_index = True)
    print(city, ' : Done !')


df_venues.head()


/home/jupyterlab/conda/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


Casablanca  : Done !
Fez  : Done !
Tangier  : Done !
Marrakesh  : Done !
Salé  : Done !
Meknes  : Done !
Rabat  : Done !
Oujda  : Done !
Kenitra  : Done !
Agadir  : Done !
Tetouan  : Done !
Temara  : Done !
Safi  : Done !
Mohammedia  : Done !
Khouribga  : Done !
El Jadida  : Done !
Beni Mellal  : Done !
Aït Melloul  : Done !
Nador  : Done !
Dar Bouazza  : Done !
Taza  : Done !
Settat  : Done !
Berrechid  : Done !
Khemisset  : Done !
Inezgane  : Done !
Ksar El Kebir  : Done !
Larache  : Done !
Guelmim  : Done !
Khenifra  : Done !
Berkane  : Done !
Taourirt  : Done !
Bouskoura  : Done !
Fquih Ben Salah  : Done !
Dcheira El Jihadia  : Done !
Oued Zem  : Done !
El Kelaa Des Sraghna  : Done !
Sidi Slimane  : Done !
Errachidia  : Done !
Guercif  : Done !
Oulad Teima  : Done !
Ben Guerir  : Done !
Tifelt  : Done !
Lqliaa  : Done !
Taroudant  : Done !
Sefrou  : Done !
Essaouira  : Done !
Fnideq  : Done !
Sidi Kacem  : Done !
Tiznit  : Done !
Tan-Tan  : Done !
Ouarzazate  : Done !
Souk El Arbaa

,City,Population,name,categories,lat,lng
0,Casablanca,"3,359,818",Casa Jose,Tapas Restaurant,33.597823,-7.615341
1,Casablanca,"3,359,818",Six PM,Hotel Bar,33.595940,-7.618684
2,Casablanca,"3,359,818",Sofitel Casablanca Tour Blanche,Hotel,33.598251,-7.613960
3,Casablanca,"3,359,818",Hyatt Regency Casablanca,Hotel,33.596195,-7.618708
4,Casablanca,"3,359,818",La Bodega,Pub,33.595220,-7.611576


In [7]:
df_categories = pd.get_dummies(df_venues['categories'])
print(df_venues.shape, df_categories.shape)
df_categories.head()


(1118, 6) (1118, 139)


,Airport,American Restaurant,Antique Shop,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,BBQ Joint,Bakery,Bar,Bay,Beach,Bed & Breakfast,Beer Garden,Big Box Store,Bistro,Boat or Ferry,Border Crossing,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Bus Station,Cafeteria,Café,Campground,Casino,Chinese Restaurant,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Creperie,Cupcake Shop,Department Store,Dessert Shop,Diner,Duty-free Shop,Eastern European Restaurant,Electronics Store,Falafel Restaurant,Farm,Fast Food Restaurant,Film Studio,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Forest,French Restaurant,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gas Station,Gastropub,General Travel,Golf Course,Grocery Store,Gym,Harbor / Marina,Historic Site,Hostel,Hotel,Hotel Bar,Hotel Pool,Housing Development,Ice Cream Shop,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Juice Bar,Kebab Restaurant,Lake,Lighthouse,Lounge,Market,Mediterranean Restaurant,Middle Eastern Restaurant,Molecular Gastronomy Restaurant,Moroccan Restaurant,Motel,Mountain,Movie Theater,Museum,Nature Preserve,Neighborhood,Nightclub,Other Great Outdoors,Other Nightlife,Park,Performing Arts Venue,Pharmacy,Pier,Pizza Place,Platform,Playground,Plaza,Pool,Pool Hall,Pub,Recreation Center,Rental Car Location,Resort,Rest Area,Restaurant,Rock Climbing Spot,Salad Place,Sandwich Place,Seafood Restaurant,Shop & Service,Shopping Mall,Snack Place,Soccer Stadium,Spa,Spanish Restaurant,Sporting Goods Shop,Steakhouse,Supermarket,Surf Spot,Sushi Restaurant,Tapas Restaurant,Taxi Stand,Tea Room,Tennis Stadium,Theater,Theme Park,Trail,Train Station,Tram Station,Travel & Transport,Vegetarian / Vegan Restaurant,Water Park,Wings Joint,Zoo
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [8]:
df_venues_categories = pd.concat([df_venues, df_categories], axis=1)
df_venues_categories.head()

,City,Population,name,categories,lat,lng,Airport,American Restaurant,Antique Shop,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,BBQ Joint,Bakery,Bar,Bay,Beach,Bed & Breakfast,Beer Garden,Big Box Store,Bistro,Boat or Ferry,Border Crossing,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Bus Station,Cafeteria,Café,Campground,Casino,Chinese Restaurant,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Creperie,Cupcake Shop,Department Store,Dessert Shop,Diner,Duty-free Shop,Eastern European Restaurant,Electronics Store,Falafel Restaurant,Farm,Fast Food Restaurant,Film Studio,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Forest,French Restaurant,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gas Station,Gastropub,General Travel,Golf Course,Grocery Store,Gym,Harbor / Marina,Historic Site,Hostel,Hotel,Hotel Bar,Hotel Pool,Housing Development,Ice Cream Shop,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Juice Bar,Kebab Restaurant,Lake,Lighthouse,Lounge,Market,Mediterranean Restaurant,Middle Eastern Restaurant,Molecular Gastronomy Restaurant,Moroccan Restaurant,Motel,Mountain,Movie Theater,Museum,Nature Preserve,Neighborhood,Nightclub,Other Great Outdoors,Other Nightlife,Park,Performing Arts Venue,Pharmacy,Pier,Pizza Place,Platform,Playground,Plaza,Pool,Pool Hall,Pub,Recreation Center,Rental Car Location,Resort,Rest Area,Restaurant,Rock Climbing Spot,Salad Place,Sandwich Place,Seafood Restaurant,Shop & Service,Shopping Mall,Snack Place,Soccer Stadium,Spa,Spanish Restaurant,Sporting Goods Shop,Steakhouse,Supermarket,Surf Spot,Sushi Restaurant,Tapas Restaurant,Taxi Stand,Tea Room,Tennis Stadium,Theater,Theme Park,Trail,Train Station,Tram Station,Travel & Transport,Vegetarian / Vegan Restaurant,Water Park,Wings Joint,Zoo
0,Casablanca,"3,359,818",Casa Jose,Tapas Restaurant,33.597823,-7.615341,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Casablanca,"3,359,818",Six PM,Hotel Bar,33.595940,-7.618684,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Casablanca,"3,359,818",Sofitel Casablanca Tour Blanche,Hotel,33.598251,-7.613960,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Casablanca,"3,359,818",Hyatt Regency Casablanca,Hotel,33.596195,-7.618708,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Casablanca,"3,359,818",La Bodega,Pub,33.595220,-7.611576,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [20]:
df_venues_categories.shape
df_venues_categories.columns[6:145]

['city']

In [63]:
data = df_venues_categories[['City','Population']+df_venues_categories.columns[6:145].tolist()].groupby(['City', 'Population'],as_index = False).sum()
data.insert(loc = 2,column = 'Latitude', value = df[['City', 'Latitude']].sort_values(by=['City']).reset_index(drop=True).iloc[:,1].values)
data.insert(loc = 3,column = 'Longitude', value = df[['City', 'Longitude']].sort_values(by=['City']).reset_index(drop=True).iloc[:,1].values)
data.insert(loc = 4,column = 'Venues Total Number', value = df_venues[['City', 'categories']].groupby(['City'],as_index = False).count().iloc[:,1].values)
data.to_csv('data.csv', sep=';')
data.head()

,City,Population,Latitude,Longitude,Venues Total Number,Airport,American Restaurant,Antique Shop,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,BBQ Joint,Bakery,Bar,Bay,Beach,Bed & Breakfast,Beer Garden,Big Box Store,Bistro,Boat or Ferry,Border Crossing,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Bus Station,Cafeteria,Café,Campground,Casino,Chinese Restaurant,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Creperie,Cupcake Shop,Department Store,Dessert Shop,Diner,Duty-free Shop,Eastern European Restaurant,Electronics Store,Falafel Restaurant,Farm,Fast Food Restaurant,Film Studio,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Forest,French Restaurant,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gas Station,Gastropub,General Travel,Golf Course,Grocery Store,Gym,Harbor / Marina,Historic Site,Hostel,Hotel,Hotel Bar,Hotel Pool,Housing Development,Ice Cream Shop,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Juice Bar,Kebab Restaurant,Lake,Lighthouse,Lounge,Market,Mediterranean Restaurant,Middle Eastern Restaurant,Molecular Gastronomy Restaurant,Moroccan Restaurant,Motel,Mountain,Movie Theater,Museum,Nature Preserve,Neighborhood,Nightclub,Other Great Outdoors,Other Nightlife,Park,Performing Arts Venue,Pharmacy,Pier,Pizza Place,Platform,Playground,Plaza,Pool,Pool Hall,Pub,Recreation Center,Rental Car Location,Resort,Rest Area,Restaurant,Rock Climbing Spot,Salad Place,Sandwich Place,Seafood Restaurant,Shop & Service,Shopping Mall,Snack Place,Soccer Stadium,Spa,Spanish Restaurant,Sporting Goods Shop,Steakhouse,Supermarket,Surf Spot,Sushi Restaurant,Tapas Restaurant,Taxi Stand,Tea Room,Tennis Stadium,Theater,Theme Park,Trail,Train Station,Tram Station,Travel & Transport,Vegetarian / Vegan Restaurant,Water Park,Wings Joint,Zoo
0,Agadir,"421,844",30.4211,-9.58306,30,0,0,0,0,0,0,0,0,1,0,0,2,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,2,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,2,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,2,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Ain Harrouda,"62,420",33.6351,-7.4508,30,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,2,0,0,0,0,0,0,0,0,2,0,0,0,0,0,2,0,0,1,0,0,0,0,1,1,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,2,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,2,0,0,0,0,0,0,0,0,0,0,3,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Al Hoceima,"56,716",35.2451,-3.93019,16,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Azrou,"54,350",33.4361,-5.22191,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Aït Melloul,"171,847",30.3381,-9.50428,29,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,6,0,0,0,0,3,0,0,0,0,0,0,1,0,0,0,0,1,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


##### As you can see the data we'll be using is given by the data frame "data". It contains the the moroccan cities theire population, latitude, longitude and the venues